## set working directory

In [1]:
import os
os.chdir('/Users/asaf/Google Drive/')
os.getcwd()

'/Users/asaf/Google Drive'

## data preprocessing

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(columnTransformer.fit_transform(X), dtype = np.str)
X = X[:, 1:]

## Splitting the dataset into the Training set and Test set

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feature Scaling

In [4]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# let's make the ANN!

In [5]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


## ANN

In [6]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'exponential'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
8000/8000 [==============================] - 2s 198us/step - loss: 0.4751 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 158us/step - loss: 0.4293 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 167us/step - loss: 0.4166 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 146us/step - loss: 0.4054 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 167us/step - loss: 0.3977 - accuracy: 0.8006
Epoch 6/100
8000/8000 [==============================] - 1s 138us/step - loss: 0.3941 - accuracy: 0.8135
Epoch 7/100
8000/8000 [==============================] - 1s 139us/step - loss: 0.3916 - accuracy: 0.8165
Epoch 8/100
8000/8000 [==============================] - 1s 148us/step - loss: 0.3871 - accuracy: 0.8211
Epoch 9/100
8000/8000 [=================

## Making predictions and evaluating the model

In [7]:

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import classification_report, balanced_accuracy_score
print(classification_report(y_test, y_pred))

precision    recall  f1-score   support

           0       0.89      0.93      0.91      1595
           1       0.68      0.56      0.61       405

    accuracy                           0.86      2000
   macro avg       0.79      0.75      0.76      2000
weighted avg       0.85      0.86      0.85      2000



## Predicting a single new observation

In [8]:
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)
new_prediction

array([[False]])

## Evaluating the ANN - Cross Validation!

In [9]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'exponential'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

#if the mean is equal +- to the accuracy from the initial ann section it's means we have low bias and low variance

## Dropout Regularization to reduce overfitting if needed

In [10]:
from keras.layers import Dropout

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

classifier.add(Dropout(p = 0.1))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'exponential'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)


Epoch 1/100
8000/8000 [==============================] - 2s 206us/step - loss: 0.4821 - accuracy: 0.7935
Epoch 2/100
8000/8000 [==============================] - 1s 165us/step - loss: 0.4339 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 164us/step - loss: 0.4240 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 155us/step - loss: 0.4134 - accuracy: 0.8016
Epoch 5/100
8000/8000 [==============================] - 1s 156us/step - loss: 0.4096 - accuracy: 0.8156
Epoch 6/100
8000/8000 [==============================] - 1s 155us/step - loss: 0.4077 - accuracy: 0.8207
Epoch 7/100
8000/8000 [==============================] - 1s 164us/step - loss: 0.4001 - accuracy: 0.8200
Epoch 8/100
8000/8000 [==============================] - 1s 164us/step - loss: 0.4003 - accuracy: 0.8224
Epoch 9/100
8000/8000 [==============================] - 1s 164us/step - loss: 0.3963 - accuracy: 0.8284
Epoch 10/100
8000/8000 [==============================]

## Tuning the ANN parameters

In [ ]:

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer,activation):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = activation, input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = activation))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)

parameters = {'batch_size': [25, 32],
                'activation': ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_